## Original code from: https://www.kaggle.com/ayuraj/submission-covid19/output

**NOTES**
1. Re-run of the code is done when submitting. Test folder is modified (hidden elements are added), therefore CAN´T submit csv file directly. 
2. Prediction must be done in Kaggle Notebook.
3. sample_submission.csv is used to verify the code extracts the correct id's from test folder.
4. This notebook sets all predicionts to "negative 1 0 0 1 1" in the study level and "none 1 0 0 1 1" in the case level.
5. ONLY FOR SUBMISSION TESTING.

# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load Submission 

In [ ]:
# Read the submisison file
sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
print(len(sub_df))
sub_df

In [ ]:
study_df = sub_df.loc[sub_df.id.str.contains('_study')]
len(study_df)

In [ ]:
image_df = sub_df.loc[sub_df.id.str.contains('_image')]
len(image_df)

In [ ]:
#Function changed from original
def prepare_test_images():
    image_id = []

    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/test')):
        for file in filenames:
            image_id.append(file.replace('.dcm', ''))

    return image_id

# Prepare Image Level Test Images

In [ ]:
image_ids = prepare_test_images()
print(f'Number of test images: {len(image_ids)}')

In [ ]:
meta_df = pd.DataFrame.from_dict({'image_id': image_ids})

# Associate image-level id with study-level ids.
# Note that a study-level might have more than one image-level ids.
for study_dir in os.listdir('../input/siim-covid19-detection/test'):
    for series in os.listdir(f'../input/siim-covid19-detection/test/{study_dir}'):
        for image in os.listdir(f'../input/siim-covid19-detection/test/{study_dir}/{series}/'):
            image_id = image[:-4]
            meta_df.loc[meta_df['image_id'] == image_id, 'study_id'] = study_dir

meta_df

In [ ]:
image_labels = []

for image_id, df in meta_df.groupby('image_id'):
    image_labels.append(f'{image_id}_image')

image_pred = ['none 1 0 0 1 1']*len(image_labels)
image_df = pd.DataFrame.from_dict({'id': image_labels, 'PredictionString': image_pred})
image_df

In [ ]:
study_labels = []

for study_id, df in meta_df.groupby('study_id'):
    study_labels.append(f'{study_id}_study')

study_pred = ['negative 1 0 0 1 1']*len(study_labels)
study_df = pd.DataFrame.from_dict({'id': study_labels, 'PredictionString': study_pred})
study_df

In [ ]:
sub_df = pd.concat([study_df, image_df])
sub_df.to_csv('submission.csv', index=False)
sub_df